In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd
from IPython.display import Image, HTML

### Definition of functions, required to proceed with web scrapped data

#### Function to convert price into number

In [2]:
def price_corr(price):
    new_price=re.sub('[$ zł]','',price)
    new_price=re.sub('[Zapytajocenę]','',new_price)
    new_price=re.sub('[*,]','.',new_price)
    
    return new_price

#### Function to convert price per m2 into number

In [3]:
def price_per_m2_corr(price):
    new_price_per_m2=re.sub('[$ zł/m²]','',price)
    new_price_per_m2=re.sub('[*,]','.',new_price_per_m2)
    
    
    return new_price_per_m2

#### Function to convert apartment size into number

In [4]:
def pow_corr(size):
    new_pow=re.sub('[$ m²]','',size)
    new_pow=re.sub('[*,]','.',new_pow)
    
    return float(new_pow)

#### Function to convert utility cost into number

In [6]:
def czynsz_corr(price):
    new_czynsz=re.sub('[$ zł]','',price)

    return new_czynsz

#### Function to read - scrap otodom.pl offer

In [7]:
def read_offer():
    
    title = ""

    while title == "":
        title = driver.find_elements_by_css_selector("h1.css-1ld8fwi")[0].text
        
        
    loc = driver.find_elements_by_css_selector("div.css-0 a")[0].text
    adress = loc.split(',')
    city=adress[0]
    off_type = driver.find_elements_by_css_selector("a.css-z43y8t")[0].text
    price = price_corr(driver.find_elements_by_css_selector("div.css-1vr19r7")[0].text)
    price_per_m2 = price_per_m2_corr(driver.find_elements_by_css_selector("div.css-18q4l99")[0].text)
    picture = driver.find_elements_by_tag_name('img')[0].get_attribute('src')
    driver.execute_script("""
        const offset = window.pageYOffset
        window.scrollTo(0, offset+475)
        """)
    # css-1i47t8k-It
    atrybuty = driver.find_elements_by_css_selector("div.css-4jf91v-Et li")
    atryb={}
    for i in atrybuty:
        i=i.text
        support = i.split(":")
        atryb[support[0]]=re.sub('^ ','',support[1])
    
    if "Powierzchnia" in atryb:
        atryb["Powierzchnia"]=pow_corr(atryb["Powierzchnia"])
        
    if "Powierzchnia działki" in atryb:
        atryb["Powierzchnia działki"]=pow_corr(atryb["Powierzchnia działki"])
        
    if "Czynsz" in atryb:
        atryb["Czynsz"]=czynsz_corr(atryb["Czynsz"])
    
    offer_dict={
        'typ_oferty':off_type,
        'nazwa':title,
        'zdjecie_link':picture,
        'lokalizacja':loc,
        'miejscowosc':city,
        'cena_PLN':price,
        'cena_za_m2':price_per_m2
    }
    
    offer_dict.update(atryb)
    
    return offer_dict
   

#### Function to select next offer

In [8]:
def go_to_offer(idx):
    elements = driver.find_elements_by_css_selector(".offer-item-title")
    elements[idx].click()


#### Function to go back to list of offers

In [9]:
def back_to_list():
    but=driver.find_element_by_css_selector("a.css-2ddjor")
    but.click()
    

#### Function to scroll down the list

In [10]:
def scroll_to_offer():
    driver.execute_script("""
    const offset = window.pageYOffset
    window.scrollTo(0, offset+348)
    """)


#### Function to go to the next page of offers

In [11]:
def new_page():
    next_page=driver.find_element_by_css_selector("span.icon-next")
    next_page.click()

### Offers' scrapping

In [12]:
options = Options()
options.headless=True
driver = webdriver.Chrome("./chromedriver",options=options)
driver.get("https://www.otodom.pl/sprzedaz/mieszkanie/krakow/?search%5Bregion_id%5D=6&search%5Bsubregion_id%5D=410&search%5Bcity_id%5D=38&search%5Border%5D=created_at_first%3Adesc")
driver.set_window_size(800, 1080)
driver.implicitly_wait(10)
driver.execute_script("""
    const offset = window.pageYOffset
    window.scrollTo(0, offset+9850)
    """)

last = int(driver.find_elements_by_css_selector("ul.pager li:nth-child(5) a")[0].text)
driver.execute_script("""
    const offset = window.pageYOffset
    window.scrollTo(0, offset-9850)
    """)

offers=[]

j=1
while j<=1:     # should be last-1
    for i in range(24):
        driver.execute_script("""
        const offset = window.pageYOffset
        window.scrollTo(0, offset+1550)
        """)
        try:
            go_to_offer(i)
        except:
            back_to_list()
            scroll_to_offer()
            
        offers.append(read_offer())
    
        back_to_list()
        scroll_to_offer()
        
    new_page()
    j+=1

driver.close()

### Save scrapped data into DataFrame

In [12]:
otodom = pd.DataFrame(offers)
otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   typ_oferty        840 non-null    object 
 1   nazwa             840 non-null    object 
 2   zdjecie_link      840 non-null    object 
 3   lokalizacja       840 non-null    object 
 4   miejscowosc       840 non-null    object 
 5   cena_PLN          840 non-null    object 
 6   cena_za_m2        840 non-null    object 
 7   Powierzchnia      228 non-null    float64
 8   Liczba pokoi      228 non-null    object 
 9   Rynek             228 non-null    object 
 10  Rodzaj zabudowy   211 non-null    object 
 11  Piętro            226 non-null    object 
 12  Liczba pięter     224 non-null    object 
 13  Materiał budynku  105 non-null    object 
 14  Okna              148 non-null    object 
 15  Ogrzewanie        193 non-null    object 
 16  Rok budowy        209 non-null    object 
 1

In [13]:
otodom.head()

,typ_oferty,nazwa,zdjecie_link,lokalizacja,miejscowosc,cena_PLN,cena_za_m2,Powierzchnia,Liczba pokoi,Rynek,...,Piętro,Liczba pięter,Materiał budynku,Okna,Ogrzewanie,Rok budowy,Stan wykończenia,Czynsz,Forma własności,Dostępne od
0,Mieszkanie na sprzedaż,Piękne studio w centrum Kazimierza.,https://www.otodom.pl/frontera/static/media/lo...,"Kraków, Stare Miasto, Kazimierz, Paulińska",Kraków,479000,14088,34.00,1,wtórny,...,parter,4,cegła,plastikowe,NaN,NaN,NaN,NaN,NaN,NaN
1,Mieszkanie na sprzedaż,"3 pokoje, po remoncie, Obozowa",https://www.otodom.pl/frontera/static/media/lo...,"Kraków, Dębniki, Ruczaj, Obozowa",Kraków,514000,9345,55.00,3,wtórny,...,1,4,cegła,plastikowe,miejskie,2003,do zamieszkania,400,pełna własność,NaN
2,Mieszkanie na sprzedaż,"Mieszkanie dwupoziomowe 60 m2, bezpośrednio",https://www.otodom.pl/frontera/static/media/lo...,"Kraków, Krowodrza, Krowodrza Górka, ul. gen. A...",Kraków,610000,10145,60.13,2,wtórny,...,7,7,NaN,plastikowe,miejskie,2006,do zamieszkania,510,pełna własność,2019-12-06
3,Mieszkanie na sprzedaż,Własne Miejsce - Piaski Wielkie 141x,https://www.otodom.pl/frontera/static/media/lo...,"Kraków, Podgórze Duchackie, Szpakowa 10",Kraków,404061,7100,56.91,3,pierwotny,...,8,11,żelbet,plastikowe,miejskie,2021,do wykończenia,NaN,pełna własność,NaN
4,Mieszkanie na sprzedaż,Własne Miejsce - Piaski Wielkie 140x,https://www.otodom.pl/frontera/static/media/lo...,"Kraków, Podgórze Duchackie, Szpakowa 10",Kraków,303630,7250,41.88,2,pierwotny,...,8,11,żelbet,plastikowe,miejskie,2021,do wykończenia,NaN,pełna własność,NaN


### Simple data management

In [14]:
otodom.drop_duplicates(inplace=True)
otodom.reset_index(drop=True,inplace=True)
otodom["cena_za_m2"] = pd.Series(otodom["cena_za_m2"], dtype= 'float64')
otodom["cena_PLN"] = pd.Series(otodom["cena_PLN"], dtype= 'float64')

### Save DataFrame to csv file

In [15]:
otodom.set_index('nazwa',inplace=True)
otodom.to_csv("otodom_file")